In [19]:
# importing libraries
import pandas as pd # data science essentials
import matplotlib.pyplot # data visualization
import seaborn as sns # enhanced data visualization
import statsmodels.formula.api as smf # linear regression (statsmodels)
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression # linear regression (scikit-learn)



# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying file name
file = './birthweight_low.xlsx'


# reading the file into Python
birthweight = pd.read_excel(io = file,
                       header = 0,
                       sheet_name = 0)



In [10]:
# imputing everything else with the median

# Male education
fill = 0
birthweight['meduc'] = birthweight['meduc'].fillna(fill)


# total number of prenatal visits 
fill = 0
birthweight['npvis'] = birthweight['npvis'].fillna(fill)


# Female Education
fill = 0
birthweight['feduc'] = birthweight['feduc'].fillna(fill)




In [11]:
# log transforming birthweight and saving it to the dataset
birthweight['log_bwght'] = np.log(birthweight['bwght']+ 0.0001)

# Adding and combining new variables
birthweight['drink_cigs'] = (birthweight['cigs']) * (birthweight['drink']/7 ) #interaction drink and cigs daily

birthweight['fage_above_34'] = np.where(birthweight['fage']> 34 , 1, 0) # Females above 34

birthweight['mage_above_34'] = np.where(birthweight['mage']> 34 , 1, 0) # Males above 34

birthweight['heavy_smoker'] = np.where(birthweight['cigs']> 11, 1, 0) # Smoking more than 11 cigs per day

birthweight['heavy_drinker'] = np.where(birthweight['drink']> 5, 1, 0) # Drinks more than 5 drinks per week



In [12]:
# preparing explanatory variable data
birthweight_data   = birthweight.drop(['log_bwght','bwght'
                               ],
                                axis = 1)


# preparing response variable data
birthweight_target = birthweight.loc[ : , 'log_bwght'] # used the log birthweight for better model



# preparing training and testing sets (all letters are lowercase)
x_train, x_test, y_train, y_test = train_test_split(
            birthweight_data,
            birthweight_target,
            test_size = 0.25,  
            random_state = 219)


# checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}              
y-side: {y_train.shape[0]}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape[0]}
""")



Training Data
-------------
X-side: (264, 22)
y-side: 264


Testing Data
------------
X-side: (88, 22)
y-side: 88



In [16]:
x_variables = [ 'drink_cigs', 'heavy_smoker', 'heavy_drinker', 'mage_above_34',
               'npvis', 'fage', 
               'cigs', 'mage', 'fage_above_34',
               'drink', 'mwhte','fblck'] # explanatory variables


# looping to make x-variables suitable for statsmodels
for val in x_variables:
    print(f"{val} +")

drink_cigs +
heavy_smoker +
heavy_drinker +
mage_above_34 +
npvis +
fage +
cigs +
mage +
fage_above_34 +
drink +
mwhte +
fblck +


In [18]:
# merging X_train and y_train so that they can be used in statsmodels
birthweight_train = pd.concat([x_train, y_train], axis = 1)


# Building the model
lm_best = smf.ols(formula =  """ log_bwght ~ drink_cigs +
heavy_smoker +
heavy_drinker +
mage_above_34 +
npvis +
fage +
cigs +
mage +
fage_above_34 +
drink +
mwhte +
fblck """,
                                data = birthweight_train)

# Fitting the model based on the data
results = lm_best.fit()



# Analyzing the summary output
print(results.summary())

# Preparing a DataFrame based the the analysis above
ols_data   = birthweight.loc[ : , x_variables]


# Preparing the target variable
birthweight_target = birthweight.loc[ : , 'log_bwght']


###############################################
## setting up more than one train-test split ##
###############################################
# FULL X-dataset (normal Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            birthweight_data,     # x-variables
            birthweight_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


# OLS p-value x-dataset (normal Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            birthweight_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

# INSTANTIATING a model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)

# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)

lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4) # using R-square
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)   # using R-square


Training_Score = lr.score(x_train_OLS, y_train_OLS).round(4) # training score for f string
Testing_Score  = lr.score(x_test_OLS, y_test_OLS).round(4) # testing score for f string
TrainTest_Gap = abs(lr_train_score - lr_test_score).round(4) # traintest gap for f string



print(f"""
Model     Train Score        Test Score        Train-Test Gap
-----     -----------        ----------         ----------
OLS       {Training_Score}              {lr_test_score}           {TrainTest_Gap}   

""")


                            OLS Regression Results                            
Dep. Variable:              log_bwght   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     76.40
Date:                Sun, 13 Mar 2022   Prob (F-statistic):           1.41e-76
Time:                        21:18:27   Log-Likelihood:                 160.44
No. Observations:                 264   AIC:                            -294.9
Df Residuals:                     251   BIC:                            -248.4
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         8.5105      0.052    162.955